# Test conversion on Niel's alpha-cyldextrin bound to 1-butylamine[+1] file!
```
/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00
```

In [77]:
%load_ext autoreload
%autoreload 2

import numpy as np
import subprocess as sp

from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *
from openforcefield.utils import mergeStructure
import parmed as pmd

from utils import create_pdb_with_conect, prune_conect
from utils import split_topology, create_host_guest_topology
from utils import create_host_mol2, convert_mol2_to_sybyl_antechamber
from utils import load_mol2, check_unique_atom_names, load_pdb
from utils import extract_water_and_ions, create_water_and_ions_parameters
from utils import check_bond_lengths
from utils import extract_dummy_atoms, create_dummy_atom_parameters
from utils import map_residues, map_atoms
from utils import copy_box_vectors
from utils import rewrite_restraints_file, rewrite_amber_input_file
from utils import color_restraints

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create a standards-compliant PDB...

In [78]:
create_pdb_with_conect(solvated_pdb='original/a-bam-p/full.crds', 
                       amber_prmtop='original/a-bam-p/full.topo', 
                       output_pdb='generated/a-bam-p/full.pdb')
prune_conect(input_pdb='full.pdb', 
             output_pdb='full_conect.pdb', 
             path='generated/a-bam-p/')

PDB file written by cpptraj.
First water residue = 160
Found first water CONECT entry at line = 8322


# Split the PDB into components and generate a topology for each of them...

In [79]:
components = split_topology(file_name='generated/a-bam-p/full.pdb')
hg_topology = create_host_guest_topology(components, host_resname='MGO', guest_resname='BAM')

# Load each topology component (from `mol2`) into separate `OEMol`s...

In [80]:
host = load_mol2(filename='original/a-bam-p/MGO.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

21 atoms in structure, 21 unique atom names.
17 atoms in structure, 17 unique atom names.


# Polymerize the host molecule and make sure there are *unique* atom types for every atom in the host...

In [81]:
create_host_mol2('generated/a-bam-p/full.pdb', 'original/a-bam-p/full.topo', 'MGO', 'generated/a-bam-p/MGO.mol2')

MOL2 file written by cpptraj.


In [82]:
convert_mol2_to_sybyl_antechamber('generated/a-bam-p/MGO.mol2', 'generated/a-bam-p/MGO-sybyl.mol2', ac_doctor=True)

MOL2 file written by antechamber.


In [83]:
host = load_mol2(filename='generated/a-bam-p/MGO-sybyl.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

126 atoms in structure, 126 unique atom names.
17 atoms in structure, 17 unique atom names.


# Parameterize the host and guest with SMIRNOFF99Frosst...

In [84]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(hg_topology.topology, molecules,
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )

# Convert the parameterized system in a ParmEd structure -- now with the SMIRNOFF99Frosst parameters...

In [85]:
hg_structure = pmd.openmm.topsystem.load_topology(hg_topology.topology, system, hg_topology.positions)

In [86]:
check_bond_lengths(hg_structure, threshold=4)

Structure looks good.


# ...and save this as an AMBER-format `prmtop` and `inpcrd`...

In [87]:
try:
    hg_structure.save('generated/a-bam-p/hg.prmtop')
except OSError:
    print('Check if the file already exists...')

Check if the file already exists...


In [88]:
try:
    hg_structure.save('generated/a-bam-p/hg.inpcrd')
except OSError:
    print('Check if the file already exists...')

Check if the file already exists...


# Use the original AMBER files to extract the waters and ions (and dummy atoms) and then parameterize those with GAFF or something else...

In [89]:
extract_water_and_ions(amber_prmtop='original/a-bam-p/full.topo',
                      amber_inpcrd='original/a-bam-p/full.crds',
                      host_residue=':MGO',
                      guest_residue=':BAM',
                      dummy=None,
                      output_pdb='generated/a-bam-p/water_ions.pdb')

Water and ion PDB file written by cpptraj.


In [90]:
create_water_and_ions_parameters(input_pdb='water_ions.pdb',
                                output_prmtop='water_ions.prmtop',
                                output_inpcrd='water_ions.inpcrd',
                                dummy_atoms=True,
                                path='generated/a-bam-p/')

Writing dummy atom `frcmod`.
Writing dummy atom `mol2`.
Water and ion parameters and coordinates written by tleap.


# Load those parameters and coordinates into a ParmEd structure... and merge with the host-guest ParmEd structure...

In [91]:
water_and_ions = pmd.amber.AmberParm('generated/a-bam-p/water_ions.prmtop', xyz='generated/a-bam-p/water_ions.inpcrd')

In [92]:
merged = mergeStructure(hg_structure, water_and_ions)

In [93]:
try:
    merged.save('generated/a-bam-p/solvated_smirnoff.prmtop')
    merged.save('generated/a-bam-p/solvated_smirnoff.inpcrd')
except:
    print('Check if file exists...')

# Now, because we lumped the dummy atoms with the water and ions, they are no longer at the beginning of the coordinate file (if we extract the dummy atoms, then [ParmEd can't merge](https://github.com/ParmEd/ParmEd/issues/952) the structures). Thus, we need to remap all restraints to new atom and residue indices...

However, we need to do the *atom* mapping on a `mol2` file and the *residue* mapping on a `pdb` file -- where we can specifically choose to *not* ignore dummy atoms -- otherwise the graph is not isomorphic...

In [94]:
reference = pmd.load_file('original/a-bam-p/full.topo', 'original/a-bam-p/full.crds')
try:
    reference.save('generated/a-bam-p/reference.pdb')
    reference.save('generated/a-bam-p/reference.mol2')
except OSError:
    print('Check if file exists...')
target = pmd.load_file('generated/a-bam-p/solvated_smirnoff.prmtop', 'generated/a-bam-p/solvated_smirnoff.inpcrd')
try:
    target.save('generated/a-bam-p/target.pdb')
    target.save('generated/a-bam-p/target.mol2')
except OSError:
    print('Check if file exists...')

Check if file exists...
Check if file exists...


In [95]:
reference_mol = load_mol2('generated/a-bam-p/reference.mol2')
target_mol = load_mol2('generated/a-bam-p/target.mol2')

In [96]:
atom_mapping = map_atoms(reference_mol, target_mol)

Determining mapping...


In [97]:
reference_mol = load_pdb('generated/a-bam-p/reference.pdb')
target_mol = load_pdb('generated/a-bam-p/target.pdb')

In [98]:
residue_mapping = map_residues(atom_mapping, reference_mol, target_mol)

# Now let's copy over the minimization input file, and replace the residue indices of the positional restraints...

In [99]:
rewrite_amber_input_file(reference_input='original/a-bam-p/therm1.in',
                        target_input='generated/a-bam-p/therm1.in',
                        reference_to_target_mapping=residue_mapping)

["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':8-10 | :7@C4 | :7@N1',


# Now let's copy over the restraint file, and replace the atom indices...

In [100]:
rewrite_restraints_file(reference_restraints='original/a-bam-p/disang.rest',
                       target_restraints='generated/a-bam-p/disang.rest',
                       reference_to_target_mapping=atom_mapping)

1,13,              → 144,10            
2,1,13,            → 145,144,10        
3,2,1,13,          → 146,145,144,10    
1,13,46,           → 144,10,43         
2,1,13,46,         → 145,144,10,43     
1,13,46,104,       → 144,10,43,101     
1,143,             → 144,140           
2,1,143,           → 145,144,140       
1,143,130,         → 144,140,127       
19,4,6,36,         → 16,1,3,33         
40,25,27,57,       → 37,22,24,54       
61,46,48,78,       → 58,43,45,75       
82,67,69,99,       → 79,64,66,96       
103,88,90,120,     → 100,85,87,117     
124,109,111,15,    → 121,106,108,12    
4,6,36,38,         → 1,3,33,35         
25,27,57,59,       → 22,24,54,56       
46,48,78,80,       → 43,45,75,77       
67,69,99,101,      → 64,66,96,98       
88,90,120,122,     → 85,87,117,119     
109,111,15,17,     → 106,108,12,14     
9,143              → 6,140             
23,143             → 20,140            
30,143             → 27,140            
44,143             → 41,140            


# Finally, copy over the box dimensions and angles, and we should be good to go...

In [101]:
copy_box_vectors(input_inpcrd='original/a-bam-p/full.crds',
                output_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd')

# Check restraints look okay in Chimera

In [31]:
color_restraints('original/a-bam-p/disang.rest', color='red',
                 suffix='niel', 
                 md_file='niel.txt',
                path='generated/a-bam-p/render-restraints/')

In [32]:
color_restraints('generated/a-bam-p/disang.rest', color='blue',
                 suffix='dave', 
                 md_file='dave.txt',
                path='generated/a-bam-p/render-restraints/')

In [103]:
pmd.tools.HMassRepartition??

Init signature: pmd.tools.HMassRepartition(input_parm, arg_list=None, *args, **kwargs)
Source:        
class HMassRepartition(Action):
    """
    This action implements hydrogen mass repartitioning in the system by
    changing the mass of each hydrogen to the desired value (the default new
    hydrogen mass is 3.024 daltons) and adjusting the mass of the atom to which
    it is bonded by the amount required to leave the total mass unchanged. By
    default, water hydrogen masses are unchanged (the SETTLE algorithm for
    implementing constraints on water molecules is analytical). Water masses can
    be repartitioned as well with the 'dowater' keyword.
    """
    usage = '[<mass>] [dowater]'
    def init(self, arg_list):
        self.changewater = arg_list.has_key('dowater')
        self.new_h_mass = arg_list.get_next_float(optional=True, default=3.024)

    def __str__(self):
        retstr = ('Repartitioning hydrogen masses to %s daltons. ' %
                  self.new_h_mass)
  

In [106]:
reference.box

array([ 38.860523,  39.115493,  56.411969,  90.      ,  90.      ,  90.      ])

In [107]:
target.box

In [108]:
target.box = reference.box

In [109]:
target.box

array([ 38.860523,  39.115493,  56.411969,  90.      ,  90.      ,  90.      ])